### Урок 8 Двухфакторный дисперсионный анализ. Факторный анализ. Логистическая регрессия#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import warnings

import math
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
from copy import deepcopy

warnings.filterwarnings('ignore')

### Задание 1.

Объясняемая переменная y зависит от двух категориальных факторов A и B, причём фактор A имеет 3 уровня, а фактор B - 4 уровня. Результаты наблюдений приведены в таблице:

    y = [[2.68, 3.29, 2.88, 4.45],
         [4.12, 4.96, 5.09, 5.22],
         [5.52, 4.50, 5.42, 5.29]]
С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную y.

In [2]:
y = np.array([[2.68, 3.29, 2.88, 4.45], [4.12, 4.96, 5.09, 5.22], [5.52, 4.50, 5.42, 5.29]])
m = y.shape[0]
k = y.shape[1]
y, m, k

(array([[2.68, 3.29, 2.88, 4.45],
        [4.12, 4.96, 5.09, 5.22],
        [5.52, 4.5 , 5.42, 5.29]]), 3, 4)

$$S_A^2 = k \cdot \displaystyle\sum_{i=1}^m \left( \overline{y_{i \ast}} - \overline{y} \right)^2, \:
S_B^2 = m \cdot \displaystyle\sum_{j=1}^k \left( \overline{y_{\ast j}} - \overline{y} \right)^2, \:
S_w^2 = \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k \left( y_{ij} - \overline{y_{i \ast}} - \overline{y_{\ast j}} + \overline{y} \right)^2.$$

In [3]:
s = 0
for i in range(m):
    s += (y[i].mean() - y.mean())**2
S_A = k * s

In [4]:
s = 0
for j in range(k):
    s += (y[:, j].mean() - y.mean())**2
S_B = m * s

In [5]:
s = 0
for i in range(m):
    for j in range(k):
        s += (y[i, j] - y[i].mean() - y[:, j].mean() + y.mean())**2
S_w = s

In [6]:
S_A, S_B, S_w

(7.8407166666666654, 1.338166666666669, 1.9298833333333327)

$$\sigma_A^2 = \dfrac{S_A^2}{m - 1}, \: \sigma_B^2 = \dfrac{S_B^2}{k - 1}, \:
\sigma_w^2 = \dfrac{S_{w}^2}{(k - 1) (m - 1)}.$$

In [7]:
var_a = S_A / (m-1)
var_b = S_B / (k-1)
var_w = S_w / ((m-1) * (k-1))
var_a, var_b, var_w

(3.9203583333333327, 0.4460555555555563, 0.32164722222222214)

In [8]:
np.var(y, axis=1)

array([0.470225  , 0.18486875, 0.16191875])

$$F_A = \dfrac{\sigma_A^2}{\sigma_w^2}, \: F_B = \dfrac{\sigma_B^2}{\sigma_w^2}.$$

In [9]:
F_A = var_a / var_w
F_B = var_b / var_w
F_A, F_B

(12.188379262995173, 1.386785038819275)

In [10]:
# Вычисляем критические значения
alpha = 0.05
n = k * m

k_a1 = m - 1
k_a2 = n - m
k_b1 = k - 1
k_b2 = n - k
F_a_crit = stats.f.ppf(1 - alpha, k_a1, k_a2)
F_b_crit = stats.f.ppf(1 - alpha, k_b1, k_b2)
F_a_crit, F_b_crit

(4.25649472909375, 4.06618055135116)

$$F_A > F_Acrit, \: F_B < F_Bcrit$$
Следовательно отвергаем гипотезу $H_0$ для фактора А и получаем существует разница между средними А, получаем что фактор А является значимым для переменной Y.
Для фактора В гипотеза подтверждается, следовательно он не является значимым.

### Задание 2.

С помощью критерия Стьюдента для двух независимых выборок проверьте гипотезу о равенстве среднего роста футболистов и хоккеистов, основываясь на результатах измерений:

    football_players = [173, 175, 180, 178, 177, 185, 183, 182]
    hockey_players = [177, 179, 180, 188, 177, 172, 171, 184, 180]

In [11]:
football_players = np.array([173, 175, 180, 178, 177, 185, 183, 182])
hockey_players = np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])
n_f = football_players.shape[0]
n_h = hockey_players.shape[0]

Проверяется гипотеза $H_0$: средние значения по двум выборкам совпадают. 

Для проверки гипотезы рассмотрим статистику
$$T = \dfrac{\overline{X_1} -\overline{X_2}}{\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}}$$

In [12]:
T = (football_players.mean() - hockey_players.mean()) / np.sqrt((football_players.var(ddof=1) / n_f) + (hockey_players.var(ddof=1) / n_h))
T

0.19928601397363732

In [13]:
alpha = 0.05
F_crit = stats.t.ppf(1 - alpha / 2, df=(n_f + n_h - 2))
F_crit

2.131449545559323

$T < Fcrit$, следовательно гипотеза $H_0$ подтверждается - средние значения по двум выборкам совпадают